In [3]:
import requests
from PIL import Image

base_url = "https://pokeapi.co/api/v2"
end_point = "/pokemon"

In [4]:
def get_pokemons():
    response = requests.get(base_url+end_point)
    if response.ok:
        return response.json()["results"]

In [5]:
def get_pokemon_sprites():
    pokemons = get_pokemons()
    sprites = {}
    for pokemon in pokemons:
        response = requests.get(pokemon["url"])
        if response.ok:
            json_data = response.json()
            sprites[pokemon["name"]] = json_data["sprites"]["front_default"]
    return sprites

In [6]:
def write_pokemon_sprites_to_files():
    sprites = get_pokemon_sprites()
    for k,v in sprites.items():
        url = v
        with open(k + '.png', 'wb') as handler:
            response = requests.get(url)
            handler.write(response.content)

In [38]:
def get_pokemon_stats():
    pokemons = get_pokemons()
    all_stats = {}
    for pokemon in pokemons:
        response = requests.get(pokemon["url"])
        if response.ok:
            json_data = response.json()
            abilities = json_data["stats"]
            stats = {}
            for ability in abilities:
                stats[ability["stat"]["name"]] = ability["base_stat"]
            all_stats[pokemon["name"]] = stats
    return all_stats

In [29]:
import psycopg2

In [31]:
def create_pokemon_table():
    with psycopg2.connect("dbname=postgres user=postgres password=mysecretpassword", port=5544) as conn:
        with conn.cursor() as cur:
            cur.execute("""CREATE TABLE pokemons (
                        "id" serial PRIMARY KEY,
                        "name" varchar,
                        "hp" int,
                        "attack" int,
                        "defense" int,
                        "special-attack" int,
                        "special-defense" int,
                        "speed" int
                        )""")
            conn.commit()

In [37]:
def insert_data():
    with psycopg2.connect("dbname=postgres user=postgres password=mysecretpassword", port=5544) as conn:
        with conn.cursor() as cur:
            pokemons = get_pokemon_stats()
            for k,v in pokemons.items():
                name = k
                hp = v["hp"]
                attack = v["attack"]
                defense = v["defense"]
                special_attack = v["special-attack"]
                special_defense = v["special-defense"]
                speed = v["speed"]
                cur.execute("""INSERT INTO pokemons (
                            "name", 
                            "hp", 
                            "attack", 
                            "defense", 
                            "special-attack", 
                            "special-defense", 
                            "speed") 
                            VALUES (%s,%s,%s,%s,%s,%s,%s);""",
                            [name, hp, attack, defense, special_attack, special_defense, speed]
                            )
            conn.commit()